# ResNet18

## 1.1. import what u need


In [1]:
import torch     
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import copy
from sklearn.metrics import accuracy_score,f1_score,roc_curve,precision_recall_curve,average_precision_score,auc
from sklearn.metrics import precision_score, recall_score, f1_score,confusion_matrix,matthews_corrcoef,roc_auc_score
import matplotlib.pyplot as plt
import torch.utils.data as Data
import numpy as np
import dataClass as MyDataClass
import pandas as pd
import random

## 1.2. data preprocessing


In [2]:
BATCH_SIZE = 32

#加载数据集并转换成torch.Tensor
# 训练集
tensor = np.load("./final-project1-j1/train_data.npy")
df = pd.read_csv("./final-project1-j1/train_data.csv")
label = list(df["label"])
# 数据reshape成(N,1,24,24)，这里加入了随机裁剪
train_tensor = []
for i in range(len(tensor)):
    train_tensor.append([])
    train_tensor[i].append([])    
    h = int(random.random() * 4)
    w = int(random.random() * 4)
    for j in range(24):
        train_tensor[i][0].append(tensor[i][(j+h)*28+w:(j+h)*28+w+24])
train_tensor = torch.Tensor(train_tensor)
train_dataset = MyDataClass.mydataset(train_tensor, label)
# 验证集
tensor = np.load("./final-project1-j1/development_data.npy")
df = pd.read_csv("./final-project1-j1/development_data.csv")
label = list(df["label"])
development_tensor = []
for i in range(len(tensor)):
    development_tensor.append([])
    development_tensor[i].append([])    
    h = int(random.random() * 4)
    w = int(random.random() * 4)
    for j in range(24):
        development_tensor[i][0].append(tensor[i][(j+h)*28+w:(j+h)*28+w+24])
development_tensor = torch.Tensor(development_tensor)
development_dataset = MyDataClass.mydataset(development_tensor, label)
# 测试集
tensor = np.load("./final-project1-j1/test.npy")
test_tensor = []
for i in range(len(tensor)):
    test_tensor.append([])
    test_tensor[i].append([])    
    h = int(random.random() * 4)
    w = int(random.random() * 4)
    for j in range(24):
        test_tensor[i][0].append(tensor[i][(j+h)*28+w:(j+h)*28+w+24])
test_dataset = torch.Tensor(test_tensor)
#加载小批次数据，即将数据集中的data分成每组batch_size的小块，shuffle指定是否随机读取
train_loader = Data.DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,shuffle=True)
development_loader = Data.DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,shuffle=True)
test_loader = Data.DataLoader(dataset=test_dataset,batch_size=BATCH_SIZE,shuffle=False)

In [3]:
class ResBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        # 经过处理后的x要与x的维度相同(尺寸和深度)
        # 如果不相同，需要添加卷积+BN来变换为同一维度
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [4]:
class Model(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(Model, self).__init__()
        self.in_planes = 32

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        
        self.layer1 = self._make_layer(block, 32, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 64, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 128, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 256, num_blocks[3], stride=2)
        self.linear = nn.Linear(256*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [19]:
model = Model(ResBlock,[2,2,2,2])#.cuda() #实例化卷积层
model.load_state_dict(torch.load("ResNetModule13_0.01_cutdata.pt"))
loss = nn.CrossEntropyLoss() #损失函数选择，交叉熵函数
optimizer = optim.SGD(model.parameters(),lr = 0.01,momentum=0.9)

## 1.2. Train


In [48]:
# 88
losses = [] 
acces = []
eval_losses = []
eval_acces = []
num_epochs = 10
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10,eta_min=4e-08)
for echo in range(num_epochs):
    train_loss = 0   #定义训练损失
    train_acc = 0    #定义训练准确度
    model.train()    #将网络转化为训练模式
    # data reload，每个epoch重新裁剪
    tensor = np.load("./final-project1-j1/train_data.npy")
    df = pd.read_csv("./final-project1-j1/train_data.csv")
    label = list(df["label"])
    train_tensor = []
    for i in range(len(tensor)):
        train_tensor.append([])
        train_tensor[i].append([])    
        h = int(random.random() * 4)
        w = int(random.random() * 4)
        for j in range(24):
            train_tensor[i][0].append(tensor[i][(j+h)*28+w:(j+h)*28+w+24])
    train_tensor = torch.Tensor(train_tensor)
    train_dataset = MyDataClass.mydataset(train_tensor, label)
    train_loader = Data.DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,shuffle=True)
    for i,(X,label) in enumerate(train_loader):     #使用枚举函数遍历train_loader
        #X = X.view(-1,784)       #X:[64,1,28,28] -> [64,784]将X向量展平
        X = Variable(X)#.cuda()          #包装tensor用于自动求梯度
        label = Variable(label)#.cuda()
        out = model(X)           #正向传播
        lossvalue = loss(out,label)         #求损失值
        optimizer.zero_grad()       #优化器梯度归零
        lossvalue.backward()    #反向转播，刷新梯度值
        optimizer.step()        #优化器运行一步，注意optimizer搜集的是model的参数
        scheduler.step()
        
        #计算损失
        train_loss += float(lossvalue)      
        #计算精确度
        _,pred = out.max(1)
        num_correct = (pred == label).sum()
        acc = int(num_correct) / X.shape[0]
        train_acc += acc

    losses.append(train_loss / len(train_loader))
    acces.append(train_acc / len(train_loader))
    print("echo:"+' ' +str(echo))
    print("lose:" + ' ' + str(train_loss / len(train_loader)))
    print("accuracy:" + ' '+str(train_acc / len(train_loader)))

echo: 0
lose: 0.014292142104447993
accuracy: 0.9956042199488491
echo: 1
lose: 0.022822222474035318
accuracy: 0.9921675191815856
echo: 2
lose: 0.02171651669028934
accuracy: 0.9927269820971867
echo: 3
lose: 0.01570033660988216
accuracy: 0.9948849104859335
echo: 4
lose: 0.01959516842792387
accuracy: 0.9932464833759591
echo: 5
lose: 0.015478323368356228
accuracy: 0.9948049872122762
echo: 6
lose: 0.014785873982854846
accuracy: 0.99528452685422
echo: 7
lose: 0.016465657324449386
accuracy: 0.9944053708439897
echo: 8
lose: 0.024005130871592905
accuracy: 0.9914881713554987
echo: 9
lose: 0.018023423264147197
accuracy: 0.9933264066496164


In [49]:
torch.save(model.state_dict(),'ResNetModule88_0.01_cutdata.pt')

## 1.3. Development

In [50]:
model.eval() #模型转化为评估模式
development_acc = 0
development_loss = 0
for X,label in development_loader:
    #X = X.view(-1,784)
    X = Variable(X)#.cuda()
    label = Variable(label)#.cuda()
    developmentout = model(X)
    developmentloss = loss(developmentout,label)
    #计算损失
    development_loss += float(developmentloss)  

    _, pred = developmentout.max(1)
    num_correct = (pred == label).sum()
    acc = int(num_correct) / X.shape[0]
    development_acc += acc
print("lose:" + ' ' + str(development_loss / len(development_loader)))
print("accuracy:" + ' '+str(development_acc / len(development_loader)))

lose: 0.009708679669534027
accuracy: 0.9973225703324808


## 1.4. Test


In [51]:
test_pred_label = []
model.eval() #模型转化为评估模式
tensor = np.load("./final-project1-j1/test.npy")
for X in test_loader:
    X = Variable(X)#.cuda()
    testout = model(X)

    _, pred = testout.max(1)
    test_pred_label += list(pred)
# 写入csv文件
for i, x in enumerate(test_pred_label):
    test_pred_label[i] = int(x)
data = {'image_id':list(range(5000)), 'label':test_pred_label}
df = pd.DataFrame(data,columns=['image_id','label'])
df.to_csv(r'submit_ResNet88_cutdata_lr0.01_randomcut_CosineDecay.csv',encoding='gbk',index=False)